In [1]:
from __future__ import division, print_function, absolute_import

seed_num = 72

import numpy as np
np.random.seed(seed_num)
import random
random.seed(seed_num)
import tensorflow as tf
tf.set_random_seed(seed_num)

import os
import sys
import glob
import math
from functools import reduce
import operator as op
from sklearn.utils import class_weight
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

dataset_folder = os.path.abspath("./individual_npzs_overlap/{0}/*.npz")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

np.set_printoptions(suppress=True)

## Parameters

In [2]:
learning_rate = 0.001
batch_size = 64
dropout = 0.75
max_pool = 2
strides = 1
input_size = 60000
output_size = 4
epochs = 10000
timesteps = 38

hidden_layer = 128
l2_beta = 0.0001
tolerance = 200

validation_session = 0
test_session = 1

label_dictionary = {'ang': 0, 'hap': 1, 'neu': 2, 'sad': 3}
onehot_dictionary = {0: 'ang', 1: 'hap', 2:'neu', 3:'sad'}

## Helper methods

In [3]:
def even_batch_generator(data, labels):
    steps = math.ceil(data.shape[0] / batch_size)
    counts = [math.floor(class_count / steps) for class_count in np.unique(train_labels, return_counts=True)[1].tolist()]
    for batch_step in range(0, steps):
        minibatch = []
        if (batch_step != steps-1):
            for count in counts:
                while (count != 0): 
                    minibatch.append()
    
    
def batch_generator(data, labels):
    steps = math.ceil(data.shape[0] / batch_size)
    for batch_step in range(0, steps):
        start = batch_size * batch_step
        end = batch_size * (batch_step + 1)
        yield data[start:end], labels[start:end], batch_step
        
def build_encoded_array(emotion_label):
    initialized_array = [0. for key in label_dictionary]
    initialized_array[label_dictionary[emotion_label]] = 1.
    return initialized_array
        
def onehot_encode(label_minibatch):
    return [build_encoded_array(emotion_label) for emotion_label in label_minibatch]

def compute_class_weights(labels):
    return np.ndarray.tolist(class_weight.compute_class_weight('balanced', np.unique(labels), labels))

def sparse_encode(label_minibatch):
    return [label_dictionary[emotion_label] for emotion_label in label_minibatch]

def onehot_decode(onehot_labels):
    return [onehot_dictionary[label.index(1.0)] for label in onehot_labels]

def weighted_accuracy(y_pred, y_true):
    #Suma vsetkych spravne urcenych viet predelena vsetkymi vetami

    correct = 0
    all = 0
    for key in y_true:
        correct += y_pred[key]
        all += y_true[key]

    return correct/all


def unweighted_accuracy(y_pred, y_true):

    #Suma poctu spravnych viet lomeno vsetky vety pre kazdu emociu (triedu) predelena poctom tried

    correct = 0
    emotions = len(y_true)
    for key in y_true:
        correct += y_pred[key]/y_true[key]

    return correct/emotions

def validation_results(current_sess, data, labels):
    predictions = current_sess.run(correct_prediction, feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    coupled_validation = list(zip(predictions, onehot_decode(labels)))
    final_dict = {'ang':0, 'hap':0, 'neu':0, 'sad':0}
    overall_dict = {'ang':0, 'hap':0, 'neu':0, 'sad':0}

    for couple in coupled_validation:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    unweighted_acc = unweighted_accuracy(final_dict, overall_dict)
    return unweighted_acc

## Load dataset: 3 sessions for training, 1 for validation, 1 for test

In [4]:
train_dataset = []
#validation_dataset = []
test_dataset = []

train_labels = []
#validation_labels = []
test_labels = []

all_sessions = {1: [], 2:[], 3:[], 4:[], 5:[]}

session_string = 'session{0}'

for i in range(1, 6):
    formatted = session_string.format(i)
    for spectrogram in glob.glob(dataset_folder.format(formatted)):
        loaded_spec = np.load(spectrogram)
        for x in loaded_spec['spectrograms']:
            if i != validation_session and i != test_session:
                train_dataset.append(x) 
         #   elif i == validation_session:
         #       validation_dataset.append(x)
            elif i == test_session:
                test_dataset.append(x)
        for x in loaded_spec['labels']:
            all_sessions[i].append(x)
            if i != validation_session and i != test_session:
                train_labels.append(x) 
          #  elif i == validation_session:
          #      validation_labels.append(x)
            elif i == test_session:
                test_labels.append(x)
        
train_dataset = np.asarray(train_dataset)
train_labels = np.asarray(train_labels)

#validation_dataset = np.asarray(validation_dataset)
#validation_labels = np.asarray(validation_labels)

test_dataset = np.asarray(test_dataset)
test_labels = np.asarray(test_labels)

In [5]:
train_data = np.zeros([len(train_dataset), train_dataset[0].shape[0], train_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(train_dataset)):
    train_data[data,:,:] = train_dataset[data]
    
#validation_data = np.zeros([len(validation_dataset), validation_dataset[0].shape[0], validation_dataset[0].shape[1]], dtype=np.uint8)
#for data in range(len(validation_dataset)):
#    validation_data[data,:,:] = validation_dataset[data]
    
test_data = np.zeros([len(test_dataset), test_dataset[0].shape[0], test_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(test_dataset)):
    test_data[data,:,:] = test_dataset[data]

In [6]:
counts = np.unique(train_labels, return_counts=True)[1]
sum_counts = sum(counts)
class_weights = [(0.25/(count/sum_counts)) for count in counts]

In [7]:
class_weights

[1.655743740795287, 2.0935754189944134, 0.602491961414791, 0.7945229681978798]

In [8]:
#np.unique(validation_labels, return_counts=True)

In [9]:
np.unique(test_labels, return_counts=True)

(array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'),
 array([ 71, 125, 670, 317], dtype=int64))

In [10]:
np.unique(train_labels, return_counts=True)

(array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'),
 array([ 679,  537, 1866, 1415], dtype=int64))

In [11]:
#validation_data = validation_data.reshape((validation_data.shape[0], input_size))
#validation_labels = onehot_encode(validation_labels)

test_data = test_data.reshape((test_data.shape[0], input_size))
test_labels = onehot_encode(test_labels)

train_data = train_data.reshape((train_data.shape[0], input_size))
train_labels = onehot_encode(train_labels)

In [12]:
train_data, train_labels = shuffle(train_data, train_labels, random_state=seed_num)

## Pipeline

In [13]:
def conv2d(to_process, weights, biases, strides=1):
    conv_out = tf.nn.conv2d(to_process, weights, strides=[1, strides, strides, 1], padding='SAME')
    bias_out = tf.nn.bias_add(conv_out, biases)
    relu_out = tf.nn.relu(bias_out)
    return relu_out

def maxpool2d(to_pool, pool_size=2):
    maxpool_out = tf.nn.max_pool(to_pool, ksize=[1, pool_size, pool_size, 1], strides=[1, pool_size, pool_size, 1], padding='SAME')
    return maxpool_out

def nn_pipeline_rnn(spectrogram, weights, biases, dropout_num, normalization_switch):
    reshaped_input = tf.reshape(spectrogram, shape=[-1, 200, 300, 1])

    first_layer_out = conv2d(reshaped_input, weights['first_layer_weights'], biases['first_layer_biases'])
    first_maxpool_out = maxpool2d(first_layer_out, pool_size=2)
    first_batch_norm = tf.layers.batch_normalization(first_maxpool_out, training=normalization_switch)

    second_layer_out = conv2d(first_batch_norm, weights['second_layer_weights'], biases['second_layer_biases'])
    second_maxpool_out = maxpool2d(second_layer_out, pool_size=2)
    second_batch_norm = tf.layers.batch_normalization(second_maxpool_out, training=normalization_switch)

    third_layer_out = conv2d(second_batch_norm, weights['third_layer_weights'], biases['third_layer_biases'])
    third_maxpool_out = maxpool2d(third_layer_out, pool_size=2)
    third_batch_norm = tf.layers.batch_normalization(third_maxpool_out, training=normalization_switch)

    interim_shape = third_batch_norm.get_shape().as_list()
    transposed = tf.transpose(third_batch_norm, perm=[0, 2, 1, 3])
    reshape_for_rnn = tf.reshape(transposed, [-1, interim_shape[2], interim_shape[1]*interim_shape[3]])
    reshape_for_rnn.set_shape([None, interim_shape[2], interim_shape[1]*interim_shape[3]])
    
    fw_cell = tf.contrib.rnn.GRUCell(hidden_layer)
    bw_cell = tf.contrib.rnn.GRUCell(hidden_layer)
    gru_fw_cell = tf.contrib.rnn.DropoutWrapper(cell=fw_cell, output_keep_prob=dropout_num, seed=seed_num)
    gru_bw_cell = tf.contrib.rnn.DropoutWrapper(cell=bw_cell, output_keep_prob=dropout_num, seed=seed_num)
    
    gru_output, _ = tf.nn.bidirectional_dynamic_rnn(gru_fw_cell, gru_bw_cell, reshape_for_rnn, dtype=tf.float32)
    gru_output_concatted = tf.concat(gru_output, axis=2)
    interim_shape_gru = tf.shape(gru_output_concatted)
    gru_flatten = tf.reshape(gru_output_concatted, [-1, interim_shape_gru[1]*interim_shape_gru[2]])
    
    fully_connected_out = tf.add(tf.matmul(gru_flatten, weights['gru_weights']), biases['gru_biases'])
    fully_connected_activation = tf.nn.relu(fully_connected_out)
    fully_connected_dropout = tf.nn.dropout(fully_connected_activation, dropout_num, seed=seed_num)
    
    prediction = tf.add(tf.matmul(fully_connected_dropout, weights['output']), biases['output'])
        
    return prediction

In [14]:
weights_rnn = {
    'first_layer_weights': tf.Variable(tf.truncated_normal([10, 15, 1, 16], seed=seed_num)),
    'second_layer_weights': tf.Variable(tf.truncated_normal([8, 10, 16, 24], seed=seed_num)),
    'third_layer_weights': tf.Variable(tf.truncated_normal([5, 8, 24, 32], seed=seed_num)),
    'gru_weights': tf.Variable(tf.truncated_normal([2*hidden_layer*timesteps, hidden_layer], seed=seed_num)),
    'output': tf.Variable(tf.truncated_normal([hidden_layer, output_size], seed=seed_num))
}

biases_rnn = {
    'first_layer_biases': tf.Variable(tf.truncated_normal([16], seed=seed_num)),
    'second_layer_biases': tf.Variable(tf.truncated_normal([24], seed=seed_num)),
    'third_layer_biases': tf.Variable(tf.truncated_normal([32], seed=seed_num)),
    'gru_biases': tf.Variable(tf.truncated_normal([hidden_layer], seed=seed_num)),
    'output': tf.Variable(tf.truncated_normal([output_size], seed=seed_num))
}

In [15]:
def loss_function(logits, labels):
    flat_logits = tf.reshape(logits, [-1, output_size])
    flat_labels = tf.reshape(labels, [-1, output_size])
    
    eps = tf.constant(value=1e-10)
    flat_logits = flat_logits + eps
    
    softmax = tf.nn.softmax(flat_logits)
    
    coeffs = tf.constant(class_weights)
    
    cross_entropy = -tf.reduce_sum(tf.multiply(flat_labels * tf.log(softmax + eps), coeffs), reduction_indices=[1])
    
    l2_loss = tf.add_n([tf.nn.l2_loss(weights_rnn[weights]) for weights in weights_rnn]) * l2_beta
    
    return tf.reduce_mean(tf.add(cross_entropy, l2_loss))

In [16]:
X = tf.placeholder(tf.float32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, output_size])

keep_prob = tf.placeholder(tf.float32)
normalization_switch = tf.placeholder(tf.bool)

logits = nn_pipeline_rnn(X, weights_rnn, biases_rnn, keep_prob, normalization_switch)
prediction = tf.nn.softmax(logits)

#weighted_logits = tf.multiply(class_weights, logits)
#weight_regularizer = tf.add_n([tf.nn.l2_loss(weights_rnn[weights]) for weights in weights_rnn]) * 0.01
#loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

loss_function = loss_function(logits, Y)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_trigger = optimizer.minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

saver = tf.train.Saver()

In [17]:
def train_network(session):
    
    best_acc = 0
    best_loss = sys.maxsize
    best_train_acc = 0
    current_tolerance = 0
    
    for epoch_step in range(1, epochs+1):
        batch_gen = batch_generator(train_data, train_labels)
        train_acc_all = []
        train_loss_all = []
        for data_minibatch, label_minibatch, current_index in batch_gen:
            sess.run(train_trigger, feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: dropout, normalization_switch: True})
            train_loss, train_acc = sess.run([loss_function, accuracy], feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: 1.0, normalization_switch: False})
            print("Train loss and acc of batch {0}: {1}, {2}".format(current_index, train_loss, train_acc))
            train_acc_all.append(train_acc)
            train_loss_all.append(train_loss)
        train_acc_whole = sum(train_acc_all)/len(train_acc_all)
        train_loss_whole = sum(train_loss_all)/len(train_loss_all)
        print("Training accuracy and loss of epoch #" + str(epoch_step) + ": {:.4f}".format(train_acc_whole) + ", {:.4f}".format(train_loss_whole))
        #loss, acc = sess.run([loss_function, accuracy], feed_dict={X: validation_data, Y: validation_labels, keep_prob: 1.0, normalization_switch: False})
        #unweighted_acc = validation_results(sess, validation_data, validation_labels)
        #print("Validation after epoch #" + str(epoch_step) + ", Validation Loss= "+ "{:.4f}".format(loss) + ", Validation Accuracy= " + "{:.3f}".format(acc) + ", Unweighted Accuracy= " + "{:.3f}".format(unweighted_acc))
        #saved_by = []

        if train_acc_whole > best_train_acc:
            best_train_acc = train_acc_whole
            saver.save(sess, "./saved_models_overlap_fasterer/best_model_bidirectional_128_l2_accuracy_overlap_faster_learning_5000eps_training_no_validation.ckpt")
            print("Saved model by train acc {0}".format(train_acc_whole))
        
        if train_loss_whole < best_loss:
            best_loss = train_loss_whole
            saver.save(sess, "./saved_models_overlap_fasterer_loss/best_model_bidirectional_128_l2_accuracy_overlap_faster_learning_5000eps_training_no_validation.ckpt")
            print("Saved model by train loss {0}".format(train_loss_whole))
      
        if train_acc_whole == 1.0:
            break

    print("Optimization Finished!")

In [ ]:
with tf.device('/gpu:0'):
    sess = tf.Session()
    sess.run(init)
    
    print('Session initialized.')
    train_network(sess)

    #print("Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={X: test_data,
    #                                  Y: test_labels,
    #                                  keep_prob: 1.0, normalization_switch: False}))

In [28]:
sess.close()

In [18]:
with tf.device('/gpu:0'):
    sess = tf.Session()

    saver.restore(sess, "./saved_models_overlap_fasterer_loss/best_model_bidirectional_128_l2_accuracy_overlap_faster_learning_5000eps_training_no_validation.ckpt")
    train_network(sess)

INFO:tensorflow:Restoring parameters from ./saved_models_overlap_fasterer_loss/best_model_bidirectional_128_l2_accuracy_overlap_faster_learning_5000eps_training_no_validation.ckpt
Train loss and acc of batch 0: 48.05868148803711, 1.0
Train loss and acc of batch 1: 48.05867385864258, 1.0
Train loss and acc of batch 2: 48.344520568847656, 0.984375
Train loss and acc of batch 3: 48.275421142578125, 0.984375
Train loss and acc of batch 4: 48.05864715576172, 1.0
Train loss and acc of batch 5: 49.407562255859375, 0.96875
Train loss and acc of batch 6: 48.56124496459961, 0.96875
Train loss and acc of batch 7: 48.05862045288086, 1.0
Train loss and acc of batch 8: 48.654319763183594, 0.984375
Train loss and acc of batch 9: 48.344451904296875, 0.984375
Train loss and acc of batch 10: 48.05859375, 1.0
Train loss and acc of batch 11: 48.0585823059082, 1.0
Train loss and acc of batch 12: 48.811798095703125, 0.984375
Train loss and acc of batch 13: 48.27532958984375, 0.984375
Train loss and acc of b

Train loss and acc of batch 65: 48.05746841430664, 1.0
Train loss and acc of batch 66: 48.05745315551758, 1.0
Train loss and acc of batch 67: 48.86991882324219, 0.96875
Train loss and acc of batch 68: 48.65314483642578, 0.984375
Train loss and acc of batch 69: 48.274192810058594, 0.984375
Train loss and acc of batch 70: 48.05742263793945, 1.0
Training accuracy and loss of epoch #2: 0.9890, 48.3888
Saved model by train loss 48.38875064043931
Train loss and acc of batch 0: 48.05741500854492, 1.0
Train loss and acc of batch 1: 48.057403564453125, 1.0
Train loss and acc of batch 2: 48.34324645996094, 0.984375
Train loss and acc of batch 3: 48.274147033691406, 0.984375
Train loss and acc of batch 4: 48.05738067626953, 1.0
Train loss and acc of batch 5: 49.40629577636719, 0.96875
Train loss and acc of batch 6: 48.559974670410156, 0.96875
Train loss and acc of batch 7: 48.057350158691406, 1.0
Train loss and acc of batch 8: 48.653045654296875, 0.984375
Train loss and acc of batch 9: 48.3431854

Train loss and acc of batch 62: 48.272987365722656, 0.984375
Train loss and acc of batch 63: 49.24761199951172, 0.96875
Train loss and acc of batch 64: 48.27296447753906, 0.984375
Train loss and acc of batch 65: 48.05619812011719, 1.0
Train loss and acc of batch 66: 48.05617904663086, 1.0
Train loss and acc of batch 67: 48.8686408996582, 0.96875
Train loss and acc of batch 68: 48.65186309814453, 0.984375
Train loss and acc of batch 69: 48.272918701171875, 0.984375
Train loss and acc of batch 70: 48.056148529052734, 1.0
Training accuracy and loss of epoch #4: 0.9890, 48.3875
Saved model by train loss 48.38747674646512
Train loss and acc of batch 0: 48.0561408996582, 1.0
Train loss and acc of batch 1: 48.056129455566406, 1.0
Train loss and acc of batch 2: 48.34197998046875, 0.984375
Train loss and acc of batch 3: 48.27287292480469, 0.984375
Train loss and acc of batch 4: 48.05610275268555, 1.0
Train loss and acc of batch 5: 49.40502166748047, 0.96875
Train loss and acc of batch 6: 48.558

Train loss and acc of batch 58: 48.05498504638672, 1.0
Train loss and acc of batch 59: 48.05498123168945, 1.0
Train loss and acc of batch 60: 48.05496597290039, 1.0
Train loss and acc of batch 61: 48.05495834350586, 1.0
Train loss and acc of batch 62: 48.27171325683594, 0.984375
Train loss and acc of batch 63: 49.246341705322266, 0.96875
Train loss and acc of batch 64: 48.271697998046875, 0.984375
Train loss and acc of batch 65: 48.0549201965332, 1.0
Train loss and acc of batch 66: 48.05491256713867, 1.0
Train loss and acc of batch 67: 48.867374420166016, 0.96875
Train loss and acc of batch 68: 48.650596618652344, 0.984375
Train loss and acc of batch 69: 48.27165222167969, 0.984375
Train loss and acc of batch 70: 48.05487823486328, 1.0
Training accuracy and loss of epoch #6: 0.9890, 48.3862
Saved model by train loss 48.386206344819406
Train loss and acc of batch 0: 48.05486297607422, 1.0
Train loss and acc of batch 1: 48.05485916137695, 1.0
Train loss and acc of batch 2: 48.34069824218

Train loss and acc of batch 54: 48.27051544189453, 0.984375
Train loss and acc of batch 55: 48.053741455078125, 1.0
Train loss and acc of batch 56: 48.053733825683594, 1.0
Train loss and acc of batch 57: 48.64942169189453, 0.984375
Train loss and acc of batch 58: 48.053714752197266, 1.0
Train loss and acc of batch 59: 48.05370330810547, 1.0
Train loss and acc of batch 60: 48.05369567871094, 1.0
Train loss and acc of batch 61: 48.053688049316406, 1.0
Train loss and acc of batch 62: 48.27043914794922, 0.984375
Train loss and acc of batch 63: 49.24507141113281, 0.96875
Train loss and acc of batch 64: 48.270423889160156, 0.984375
Train loss and acc of batch 65: 48.05364990234375, 1.0
Train loss and acc of batch 66: 48.05363845825195, 1.0
Train loss and acc of batch 67: 48.8661003112793, 0.96875
Train loss and acc of batch 68: 48.649322509765625, 0.984375
Train loss and acc of batch 69: 48.2703857421875, 0.984375
Train loss and acc of batch 70: 48.05360794067383, 1.0
Training accuracy and l

...

Train loss and acc of batch 12: 48.43445587158203, 0.984375
Train loss and acc of batch 13: 47.897987365722656, 0.984375
Train loss and acc of batch 14: 47.897979736328125, 0.984375
Train loss and acc of batch 15: 48.27690887451172, 0.984375
Train loss and acc of batch 16: 48.27690124511719, 0.984375
Train loss and acc of batch 17: 48.43441390991211, 0.984375
Train loss and acc of batch 18: 48.5627326965332, 0.96875
Train loss and acc of batch 19: 47.681175231933594, 1.0
Train loss and acc of batch 20: 47.6811637878418, 1.0
Train loss and acc of batch 21: 48.27685546875, 0.984375
Train loss and acc of batch 22: 48.27684020996094, 0.984375
Train loss and acc of batch 23: 47.68113708496094, 1.0
Train loss and acc of batch 24: 48.276824951171875, 0.984375
Train loss and acc of batch 25: 47.681121826171875, 1.0
Train loss and acc of batch 26: 47.68111038208008, 1.0
Train loss and acc of batch 27: 47.68109893798828, 1.0
Train loss and acc of batch 28: 47.68109130859375, 1.0
Train loss and a

Train loss and acc of batch 9: 47.965850830078125, 0.984375
Train loss and acc of batch 10: 47.679996490478516, 1.0
Train loss and acc of batch 11: 47.67998504638672, 1.0
Train loss and acc of batch 12: 48.433197021484375, 0.984375
Train loss and acc of batch 13: 47.89673614501953, 0.984375
Train loss and acc of batch 14: 47.896728515625, 0.984375
Train loss and acc of batch 15: 48.27565002441406, 0.984375
Train loss and acc of batch 16: 48.27564239501953, 0.984375
Train loss and acc of batch 17: 48.43315505981445, 0.984375
Train loss and acc of batch 18: 48.56147766113281, 0.96875
Train loss and acc of batch 19: 47.67991638183594, 1.0
Train loss and acc of batch 20: 47.679908752441406, 1.0
Train loss and acc of batch 21: 48.275596618652344, 0.984375
Train loss and acc of batch 22: 48.27558898925781, 0.984375
Train loss and acc of batch 23: 47.67987823486328, 1.0
Train loss and acc of batch 24: 48.27557373046875, 0.984375
Train loss and acc of batch 25: 47.67986297607422, 1.0
Train los

Train loss and acc of batch 6: 48.18138885498047, 0.96875
Train loss and acc of batch 7: 47.67876052856445, 1.0
Train loss and acc of batch 8: 48.27445983886719, 0.984375
Train loss and acc of batch 9: 47.96460723876953, 0.984375
Train loss and acc of batch 10: 47.67873764038086, 1.0
Train loss and acc of batch 11: 47.67873001098633, 1.0
Train loss and acc of batch 12: 48.431941986083984, 0.984375
Train loss and acc of batch 13: 47.895477294921875, 0.984375
Train loss and acc of batch 14: 47.895469665527344, 0.984375
Train loss and acc of batch 15: 48.274391174316406, 0.984375
Train loss and acc of batch 16: 48.274383544921875, 0.984375
Train loss and acc of batch 17: 48.43190002441406, 0.984375
Train loss and acc of batch 18: 48.56022262573242, 0.96875
Train loss and acc of batch 19: 47.67865753173828, 1.0
Train loss and acc of batch 20: 47.67864990234375, 1.0
Train loss and acc of batch 21: 48.27433776855469, 0.984375
Train loss and acc of batch 22: 48.274330139160156, 0.984375
Train

Train loss and acc of batch 2: 47.677547454833984, 1.0
Train loss and acc of batch 3: 47.89429473876953, 0.984375
Train loss and acc of batch 4: 47.677528381347656, 1.0
Train loss and acc of batch 5: 49.026451110839844, 0.96875
Train loss and acc of batch 6: 48.18013381958008, 0.96875
Train loss and acc of batch 7: 47.67750930786133, 1.0
Train loss and acc of batch 8: 48.273193359375, 0.984375
Train loss and acc of batch 9: 47.96333312988281, 0.984375
Train loss and acc of batch 10: 47.6774787902832, 1.0
Train loss and acc of batch 11: 47.67747497558594, 1.0
Train loss and acc of batch 12: 48.43068313598633, 0.984375
Train loss and acc of batch 13: 47.89421844482422, 0.984375
Train loss and acc of batch 14: 47.89421081542969, 0.984375
Train loss and acc of batch 15: 48.27313995361328, 0.984375
Train loss and acc of batch 16: 48.27312469482422, 0.984375


KeyboardInterrupt: 

In [37]:
sess = tf.Session()

saver.restore(sess, "./saved_models_overlap_fasterer/best_model_bidirectional_128_l2_accuracy_overlap_faster_learning_5000eps_training_no_validation.ckpt")


INFO:tensorflow:Restoring parameters from ./saved_models_overlap_fasterer/best_model_bidirectional_128_l2_accuracy_overlap_faster_learning_5000eps_training_no_validation.ckpt


In [38]:
def get_confusion_matrix(session, data, labels):
    predictions = sess.run(tf.argmax(prediction, 1), feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    true_labels = [onehot.index(1.0) for onehot in labels]
    return confusion_matrix(predictions, true_labels)

def get_final_results(session, data, labels):
    final_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    overall_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    predictions = session.run(correct_prediction, feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    coupled = list(zip(predictions, onehot_decode(labels)))
    
    for couple in coupled:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    
    return final_dict, overall_dict

def get_final_results_glued(session, labels, correct):
    final_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    overall_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    coupled = list(zip(correct, onehot_decode(labels)))
    
    for couple in coupled:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    
    return final_dict, overall_dict

In [39]:
final_preds = np.asarray([])
final_correct_preds = np.asarray([])
final_probs = np.empty((0, 4))
final_probs_softmax = np.empty((0, 4))


midpoint = math.ceil(len(test_labels)/2)
for i in range(0, 2):
    predictions_test = sess.run(correct_prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    probabilities_test = sess.run(prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    predictions_test_confusion = sess.run(tf.argmax(prediction, 1), feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    predictions_test_softmax = sess.run(prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})

    final_correct_preds = np.concatenate((final_correct_preds, predictions_test))
    final_probs = np.concatenate((final_probs, probabilities_test))
    final_preds = np.concatenate((final_preds, predictions_test_confusion))
    final_probs_softmax = np.concatenate((final_probs_softmax, predictions_test_softmax))

In [40]:
confusion_matrix(final_preds, [onehot.index(1.0) for onehot in test_labels])

array([[ 40,  21,  48,   5],
       [  8,  27,  67,   4],
       [ 16,  76, 430,  70],
       [  7,   1, 125, 238]], dtype=int64)

In [41]:
final_dict_test, overall_dict_test = get_final_results_glued(sess, test_labels, final_correct_preds)

In [42]:
print(final_dict_test)
print(overall_dict_test)

{'ang': 40, 'hap': 27, 'neu': 430, 'sad': 238}
{'ang': 71, 'hap': 125, 'neu': 670, 'sad': 317}


In [43]:
print(unweighted_accuracy(final_dict_test, overall_dict_test))
print(weighted_accuracy(final_dict_test, overall_dict_test))

0.5429899924998458
0.621301775147929


In [44]:
print(precision_recall_fscore_support(final_preds, [onehot.index(1.0) for onehot in test_labels], average='macro')[0:2])

(0.5429899924998458, 0.49336373985703674)


In [45]:
np.set_printoptions(threshold=np.nan)

print(final_probs)

[[0.         0.         1.         0.        ]
 [0.         0.         1.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         

In [27]:
final_probs_softmax

array([[0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        ,